## Импорт и парсинг данных

### Задание: получение данных по API

Изучите API Геокодера Яндекса

http://tech.yandex.ru/maps/geocoder/doc/desc/concepts/input_params-docpage/

и получите ключ API для него в кабинете разработчика.
Выполните запрос к API и узнайте долготу точки на карте (Point) для города Самара.
Внимание: активация ключа Геокодера Яндекса может занимать несколько часов (до суток)

###############################################################################################################################

Импортируем библиотеки.

In [3]:
import requests
import json
import pandas as pd

Создаём запрос к API. В запросе задаём полученный ключ, геоданные и желаемый формат данных. Загружаем данные в датафрейм. 

In [7]:
request = requests.get("https://geocode-maps.yandex.ru/1.x/?apikey=b5b6c0e2-7cf8-4ff0-aa6d-36293d09197e&geocode=Самара&format=json")
df = pd.DataFrame(json.loads(request.content))
df

,response
GeoObjectCollection,{'metaDataProperty': {'GeocoderResponseMetaDat...


Выбираем в датафрейме нужные нам данные. Координаты указаны в формате 'pos': 'долгота широта'

In [29]:
df.response['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']

{'pos': '50.100193 53.195873'}

### Задание: получение котировок акций

Получите данные по котировкам акций со страницы:

http://mfd.ru/marketdata/?id=5&group=16&mode=3&sortHeader=name&sortOrder=1&selectedDate=01.11.2019

и найдите, по какому тикеру был максимальный рост числа сделок (в процентах) за 1 ноября 2019 года.

###############################################################################################################################

Импортируем библиотеки.

In [62]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Создаём запрос к сайту и парсим данные. Для этого ищем теги с таблицами и определёнными id. Создаём список с данными, опуская теги. Затем конвертируем полученные данные в датафрейм, названия колонок берём с сайта.

In [75]:
request = requests.get('http://mfd.ru/marketdata/?id=5&group=16&mode=3&sortHeader=name&sortOrder=1&selectedDate=01.11.2019')
html = BeautifulSoup(request.content)
data = []
table = html.find('table', {'id':'marketDataList'})
for tr in table.find_all('tr'):
    data.append([td.get_text(strip = True) for td in tr.find_all('td')])
data = pd.DataFrame(data, columns = ['Тикер', 'Время', 'Сделки', 'C/рост', 'С/%', 'Пред закр', 'Откр', 'мин', 'макс', 'срвзв', 'шт', 'руб', 'Кол-во сделок']).dropna(axis = 0).reset_index().drop('index', axis = 1)
data.head()

,Тикер,Время,Сделки,C/рост,С/%,Пред закр,Откр,мин,макс,срвзв,шт,руб,Кол-во сделок
0,+МосЭнерго,01.11.2019,2.29,+0.0215,+0.95%,2.2685,2.28,2.268,2.2925,2.281,4 299 000,9 802 392.5,517
1,AGRO-гдр,01.11.2019,677.4,−4,−0.59%,681.4,681.4,672,687.6,678.2,19 879,13 481 438.4,1 152
2,ENPL-гдр,01.11.2019,591,+3.6,+0.61%,587.4,590.2,578.2,595,591,90 989,53 760 851.6,605
3,FIVE-гдр,01.11.2019,2 168,+21,+0.98%,2 147,2 142.5,2 131,2 197,2 169,235 353,510 463 327.5,4 858
4,FXAU ETF,01.11.2019,1 985,−2,−0.10%,1 987,1 993,1 980,2 000,1 992,490,976 277,99


In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Тикер          371 non-null    object
 1   Время          371 non-null    object
 2   Сделки         371 non-null    object
 3   C/рост         371 non-null    object
 4   С/%            371 non-null    object
 5   Пред закр      371 non-null    object
 6   Откр           371 non-null    object
 7   мин            371 non-null    object
 8   макс           371 non-null    object
 9   срвзв          371 non-null    object
 10  шт             371 non-null    object
 11  руб            371 non-null    object
 12  Кол-во сделок  371 non-null    object
dtypes: object(13)
memory usage: 37.8+ KB


Создаём вспомогательную функцию для изменения типа необходимой нам колонки на float.

In [77]:
def str_to_float(s):
    if '+' in s:
        s = float(s[1:-1])
    elif '-' in s or '−' in s:
        s = -float(s[1:-1])
    elif s == '':
        s = 0
    else:
        s = float(s[:-1])
    return s
data['С/%'] = data['С/%'].apply(str_to_float)

Сортируем данные по максимальному росту числа сделок (в процентах) и выводим на экран максимальное значение за 1 ноября 2019 года.

In [80]:
data.sort_values('С/%', ascending = False).head(1)

,Тикер,Время,Сделки,C/рост,С/%,Пред закр,Откр,мин,макс,срвзв,шт,руб,Кол-во сделок
113,ИКРУСС-ИНВ,01.11.2019,33.3,+3.3,11.0,30,29.9,29.9,36.9,32.2,32 300,1 040 630,168


### Задание: парсинг интернет-магазина

Используя парсинг данных с маркетплейса beru.ru, найдите, на сколько литров отличается общий объем холодильников Саратов 263 и Саратов 452?

Для парсинга можно использовать зеркало страницы beru.ru с результатами для холодильников Саратов по адресу:

http://video.ittensive.com/data/018-python-advanced/beru.ru/

На сколько литров отличается общий объем холодильников?

###############################################################################################################################

Импортируем библиотеки.

In [8]:
import requests
import numpy as np
from bs4 import BeautifulSoup

Создаём функцию-помощник для нахождения ссылок в html файле по тэгам. Прописываем имя User-Agent для парсера. Делаем запрос к зеркалу страницы. Находим нужные нам ссылки. Создаём ещё два запроса к странице с полученными ссылками. Из полученных ссылок извлекаем данные по объёму холодильников и приводим их к числовому значению. Вычисляем разницу в объемах холодильников.

In [11]:
def find_link(html, tag, _class = None):
    links = html.find_all(tag, _class)
    return links

headers = {'User-Agent': 'ittensive-python-courses/1.0 (+https://www.ittensive.com)'}
r = requests.get('http://video.ittensive.com/data/018-python-advanced/beru.ru/', headers=headers)
html = BeautifulSoup(r.content)
links = find_link(html, 'a', {'class': '_3ioN70chUh Usp3kX1MNT _3Uc73lzxcf'})
link_263 = ''
link_452 = ''
for link in links:
    if str(link).find('Саратов 263') > -1:
        link_263 = link['href']
    if str(link).find('Саратов 452') > -1:
        link_452 = link['href']
        
r_263 = requests.get('http://video.ittensive.com/data/018-python-advanced/beru.ru/' + link_263)
r_452 = requests.get('http://video.ittensive.com/data/018-python-advanced/beru.ru/' + link_452)
html_263 = BeautifulSoup(r_263.content)
html_452 = BeautifulSoup(r_452.content)
volume_263_link = find_link(html_263, 'span', {'class': '_112Tad-7AP'})
volume_452_link = find_link(html_452, 'span', {'class': '_112Tad-7AP'})
volume_263 = int(''.join(i for i in volume_263_link[2].get_text() if i.isdigit()))
volume_452 = int(''.join(i for i in volume_452_link[2].get_text() if i.isdigit()))

diff = np.maximum(volume_263, volume_452) - np.minimum(volume_263, volume_452)
print(f'Количество литров на которые отличается общий объём холодильников: {diff}')

Количество литров на которые отличается общий объём холодильников: 73


### Задание: загрузка результатов в БД

Соберите данные о моделях холодильников Саратов с маркетплейса beru.ru: URL, название, цена, размеры, общий объем, объем холодильной камеры.

Создайте соответствующие таблицы в SQLite базе данных и загрузите полученные данные в таблицу beru_goods.
Для парсинга можно использовать зеркало страницы beru.ru с результатами для холодильников Саратов по адресу:

http://video.ittensive.com/data/018-python-advanced/beru.ru/

Какая ширина у холодильника Саратов 264 в см?

###############################################################################################################################

Импортируем необходимые библиотеки.

In [15]:
import sqlite3
import requests
from bs4 import BeautifulSoup

Создаём вспомогательные функции: для извлечения чисел из текста, для нахождения ссылок по тэгам, для нахождения значений для занесения в базу данных - имя ссылки(link), наименование товара(title), его цена(price), ширина(w), глубина(d), высота(h), объем(v), объём морозильной камеры(f).

In [16]:
def find_number(text):
    return int("0" + "".join(i for i in text if i.isdigit()))

def find_link(html, tag, _class = None):
    links = html.find_all(tag, _class)
    return links

def find_info(link):
    r = requests.get('http://video.ittensive.com/data/018-python-advanced/beru.ru/' + link)
    html = BeautifulSoup(r.content)
    title = find_link(html, 'h', {'class': '_3TfWusA7bt'}).get_text()
    price = find_number(find_link(html, 'span', {'data-tid': 'c3eaad93'}).get_text())
    tags = find_number(html, 'span', {'class': '_112Tad-7AP'})
    w, d, h, v, f = 0, 0, 0, 0, 0 
    for tag in tags:
        tag = tag.get_text()
        if tag.find('ШхВхГ') > -1:
            dims = tag.split(':')[1].split('х')
            w = float(dims[0])
            d = float(dims[1])
            h = float(dims[2].split(' ')[0])
        if tag.find('общий объем') > -1:
            v = find_number(tag)
        if tag.find('объем холодильной камеры') > -1:
            f = find_number(tag)
    return [link, title, price, w, d, h, v, f]

Создаём запрос к зеркалу страницы. Создаём пустой список для его дальнейшего занесения в базу данных. Ищем ссылки на нужный товар. Подключаемся к базе данных. Создаём таблицу с заданными параметрами и значениями по умолчанию. Передаём данные из списка в созданную таблицу. Выбираем из базы данных холодильник Саратов 264 и выводим на экран его название и объём.

In [17]:
r = requests.get('http://video.ittensive.com/data/018-python-advanced/beru.ru/')
html = BeautifulSoup(r.content)
links = find_link(html, 'a', {'class': '_3ioN70chUh Usp3kX1MNT _3Uc73lzxcf'})
data = []
for link in links:
    if link['href'] and link.get_text().find('Саратов') > -1:
        data.append(find_data(link['href']))

conn = sqlite3.connect("C:/Users/dzerg/Desktop/sqlite/data.db3")
db = conn.cursor()
db.execute("""DROP TABLE beru_goods""")
db.execute("""CREATE TABLE beru_goods
            (id INTEGER PRIMARY KEY AUTOINCREMENT not null,
            url text,
            title text default '',
            price INTEGER default 0,
            width FLOAT default 0.0,
            depth FLOAT default 0.0,
            height FLOAT default 0.0,
            volume INTEGER default 0,
            freezer INTEGER default 0)""")
conn.commit()
db.executemany("""INSERT INTO beru_goods (url, title, price, width, depth, height, volume, freezer)
           VALUES (?, ?, ?, ?, ?, ?, ?, ?)""", data)
conn.commit()
print (db.execute("SELECT title, width FROM beru_goods WHERE title = 'Холодильник Саратов 264 (КШД-150/30)'").fetchall())
db.close()

[('Холодильник Саратов 264 (КШД-150/30)', 48.0)]
